In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO


# Load a model
model = YOLO("yolo11n-pose.pt")

In [ ]:
# Train the model
results = model.train(data="hand-keypoints.yaml", epochs=15, imgsz=640)

In [ ]:
import cv2
cap = cv2.VideoCapture(1)

if not cap.isOpened():
    print("Error: Unable to access the webcam.")
    exit()

# Main loop to process frames from the webcam
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Unable to read frame from the webcam.")
        break

    # Resize frame for better performance
    frame_resized = cv2.resize(frame, (640, 480))

    # Perform inference
    results = model(frame_resized)

    # Extract keypoints from results
    detections = results.xyxy[0].cpu().numpy()  # Format: [x1, y1, x2, y2, confidence, class]

    # Draw keypoints on the frame
    for det in detections:
        x1, y1, x2, y2, conf, cls = map(int, det[:6])
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2  # Center of the box
        cv2.circle(frame_resized, (cx, cy), 5, (0, 255, 0), -1)  # Draw the center keypoint

    # Display the frame with keypoints
    cv2.imshow("Hand Keypoints Detection", frame_resized)

    # Break loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
